In [ ]:
# In first cell I hardcoded "golden standard" - tags which I've picked from the wordnet which I know they are correct.
# It is essential for my evaluation. In actual process it won't be needed or it can be changed to wanted IDs.
# This cell WOULDN'T be in here during sharp process.

GOLD_STANDARD = {
    # 1. "Na kole..."
    "kolo": [
        "cswn-02834778-n", # bicycle/bike (vozidlo) - PRIORITNÍ
        "cswn-02974003-n"  # wheel (kolo jako část) - TECHNICKY SPRÁVNĚ (jedeš po kolech)
    ],

    # 2. "...se mu..."
    "on": [
        "cswn-77000046-n", # zájmeno (častý mapping v tomto WN)
        "oth"              # gramatické slovo
    ],

    # 3. "...udělalo..." (stalo se)
    "udělat": [
        "cswn-02561995-v", # get done/make
        "cswn-00344418-v", # happen/occur (stát se)
        "cswn-01712704-v"  # carry out/perform (někdy modely volí toto)
    ],

    # 4. "...nevolno..." (stav)
    "nevolno": ["oth"], # Ve WN chybí příslovce "ill/nauseous", takže OTH je správně

    # 5. "...nesedl..." (metafora)
    "nesedl": ["oth"], # Sloveso "sednout" ve smyslu "vyhovovat" tam není.

    # 6. "...poslední..."
    "poslední": [
        "cswn-01579128-a", # last/final (konečný v řadě)
        "cswn-01578312-a"  # ultimate/fundamental (někdy se překrývá)
    ],

    # 7. "...panák..." (alkohol) - TADY JE ČASTÝ SPOR
    "panák": [
        "cswn-07826283-n", # small drink/dram (štamprle) - NEJPŘESNĚJŠÍ
        "cswn-13773250-n", # shot/liquor (nápoj)
        "cswn-04249882-n"  # glass (sklenička) - metonymie, uznáváme
    ],

    # 8. "Cestou..."
    "cesta": [
        "cswn-04096066-n", # way/road (komunikace)
        "cswn-04564698-n", # path/way (trasa)
        "cswn-00277962-n"  # journey (děj)
    ],

    # 9. "...domů..."
    "dům": [
        "cswn-03544360-n", # dwelling/house (budova)
        "cswn-08078020-n", # household/family (domov)
        "loc"              # Location tag
    ],

    # 10. "...si všiml..."
    "všimnout": ["cswn-02138549-v"], # notice/perceive

    # 11. "...na háku..."
    "hák": [
        "cswn-03532672-n", # mechanical device (hák jeřábu)
        "cswn-03532342-n"  # curved implement (obecný hák)
    ],

    # 12. "...visí..."
    "viset": ["cswn-01482075-v"], # be suspended

    # 13. "...těžký..."
    "těžký": [
        "cswn-01184932-a", # heavy/weighty (fyzická váha)
        "cswn-00837415-a"  # difficult/heavy (těžká práce) - v kontextu nákladu bereme spíš váhu
    ],

    # 14. "...náklad..."
    "náklad": [
        "cswn-02964389-n", # cargo/freight (zboží)
        "cswn-00017122-n"  # burden/load (obecná zátěž)
    ],

    # 15. "...kouřili..." (tabák)
    "kouřit": ["cswn-01198101-v"], # smoke (inhale tobacco)

    # 16. "...dělníci."
    "dělník": ["cswn-10791221-n"], # workman

    # 17. "Nakonec..."
    "nakonec": ["cswn-00047641-r"], # finally/eventually

    # 18. "...musel..."
    "muset": [
        "cswn-02613229-v", # must/necessity (sloveso)
        "cswn-09363970-n"  # must (podstatné jméno "nutnost") - modely to pletou, uznejme obojí
    ],

    # 19. "...vyměnit..." (simplemma vrací 'měnit')
    "měnit": [
        "cswn-00123170-v", # change/replace (nahradit)
        "cswn-02257370-v"  # exchange (výměna kus za kus)
    ],

    # 20. "...duši..." (pneumatika)
    "duše": [
        "cswn-10627082-n", # inner tube (duše v kole) - PRIORITNÍ
        "cswn-03573005-n"  # rubber tube (materiál)
    ],

    # 21. "...úplně..."
    "úplně": [
        "cswn-00008007-r", # completely
        "oth"              # příslovce
    ],

    # 22. "...prázdná."
    "prázdný": ["cswn-01087977-a"] # empty/void
}


In [ ]:
import wn
import ollama
import pandas as pd
import simplemma
import re
import os
import time
import warnings
from datetime import datetime


# Key = LEMMA (jby simplemma)
# Value = List of correct IDs (all synonyms, which are making sense)

# | SPECIAL TAGS | 
#                      Logic for LLM tagging fallbacks adapted from the NTUMC project.
#                      URL: https://github.com/bond-lab/NTUMC/blob/refactored-tagdb/ntumc/taggers/tag-llm.py
#                      Author/Lab: Bond Lab (Francis Bond et al.)

SPECIAL_TAGS = {
    'per': 'name of a person not in wordnet (e.g., Irena, Jiří Novák)',
    'org': 'name of an organization not in wordnet (e.g., IBM, Google)',
    'dat': 'date/time that is not in wordnet (e.g., 2pm, Monday, 16:17)',
    'loc': 'name of a place not in wordnet (e.g., Olomouc, Prague)',
    'oth': 'other name not in wordnet (e.g., Thinkpad, iPhone)',
    'year': 'name of a year not in wordnet (e.g., 1967, 2024)',
    'num': 'number not in wordnet (e.g., 42, three)',
    'e': 'the word was not tokenized or lemmatized correctly',
    'w': 'wordnet does not have the correct sense (e.g., Python as a programming language)',
    'x': 'closed class word (preposition, pronoun, punctuation) or part of a multiword expression'
}

# | STOP WORDS | To optimalize speedness I am skipping stopwords. This saved me a lot of time.
STOP_WORDS_LIST = [
    "a", "aby", "abych", "abychom", "abys", "abyste", "ahoj", "aj", "ale", "anebo", "ani", "aniž", "ano", "asi", "aspoň", "atd", "atp", "avšak", "az", "ačkoli", "až", "blízko", "bohužel", "brzo", "bude", "budem", "budeme", "budes", "budeš", "budete", "budou", "budu", "by", "bych", "bychom", "byl", "byla", "byli", "bylo", "byly", "bys", "byste", "byt", "během", "bez", "beze", "být", "ci", "clanek", "clanku", "clanky", "co", "copak", "coz", "což", "cz", "chce", "chceme", "chceš", "chcete", "chci", "chtějí", "chtít", "chut'", "chuti", "daleko", "dalsi", "další", "den", "deset", "design", "devatenáct", "devět", "dnes", "do", "dobrý", "docela", "dva", "dvacet", "dvanáct", "dvě", "dál", "dále", "děkovat", "děkujeme", "děkuji", "dělá", "dělají", "dělám", "děláme", "děláš", "děláte",  "email", "ho", "hodně", "i", "jak", "jakmile", "jako", "jakož", "jaký", "jde", "je", "jeden", "jedenáct", "jedna", "jedno", "jednou", "jedou", "jeho", "jehož", "jej", "jeji", "jejich", "její", "jelikož", "jemu", "jen", "jenom", "jenž", "jeste", "jestli", "jestliže", "ještě", "jež", "ji", "jich", "jimi", "jinak", "jine", "jiné", "jiz", "již", "jsem", "jses", "jseš", "jsi", "jsme", "jsou", "jste", "já", "jí", "jím", "jíž", "jšte", "k", "kam", "každý", "kde", "kdo", "kdy", "kdyby", "kdybych", "kdybychom", "kdybys", "kdybyste", "kdyz", "když", "ke", "kolik", "kolem", "kromě", "ktera", "ktere", "kteri", "kterou", "ktery", "která", "které", "který", "kteři", "kteří", "ku", "kvůli", "ma", "mají", "mate", "me", "mezi", "mi", "mit", "mne", "mnou", "mně", "moc", "mohl", "mohou", "moje", "moji", "možná", "muj", "musí", "muze", "my", "má", "málo", "mám", "máme", "máte", "máš", "mé", "mě", "mí", "mimo", "místo", "mít", "můj", "může", "na", "nad", "nade", "nam", "napiste", "napište", "naproti", "nas", "nasi", "načež", "naše", "naši", "ne", "nebo", "nebyl", "nebyla", "nebyli", "nebyly", "nechť", "nedělá", "nedělají", "nedělám", "neděláme", "neděláš", "neděláte", "neg", "nejsi", "nejsou", "nemají", "nemáme", "nemáte", "neměl", "neni", "není", "nestačí", "nevadí", "nez", "než", "nic", "nich", "nimi", "nove", "novy", "nové", "nový", "nula", "ná", "nám", "námi", "nás", "náš", "ně", "něco", "nějak", "někde", "někdo", "němu", "němuž", "ní", "ním", "nýbrž", "nyní", "o", "od", "ode", "on", "ona", "oni", "ono", "ony", "osm", "osmnáct", "pak", "patnáct", "po", "pod", "podle", "pokud", "potom", "pouze", "pozdě", "pořád", "prave", "pravé", "pred", "pres", "pri", "pro", "proc", "prosím", "prostě", "proti", "proto", "protoze", "protože", "proč", "prvni", "první", "práve", "prý", "pta", "pět", "před", "přede", "přes", "přese", "při", "přičemž", "re", "rovně", "rovněž", "s", "se", "sedm", "sedmnáct", "si", "sice", "skoro", "skrz", "smějí", "smí", "snad", "spolu", "sta", "sto", "strana", "sté", "sve", "svych", "svym", "svymi", "své", "svůj", "svých", "svým", "svými", "sem", "ta", "tady", "tahle", "tak", "take", "takhle", "taky", "takze", "také", "takže", "tam", "tamhle", "tamhleto", "tamto", "tato", "te", "tebe", "tebou", "ted'", "tedy", "tema", "ten", "tenhle", "tento", "teto", "teď", "ti", "tim", "timto", "tipy", "tisíc", "tisíce", "to", "tobě", "tohle", "toho", "tohoto", "tom", "tomto", "tomu", "tomuto", "toto", "trošku", "tu", "tudíž", "tuto", "tvoje", "tvá", "tvé", "tvůj", "ty", "tyto", "téma", "této", "tě", "těm", "těma", "těmu", "tím", "tímto", "třeba", "tři", "třináct", "u", "určitě", "uz", "už", "v", "vam", "vas", "vase", "vaše", "vaši", "ve", "večer", "vedle", "vice", "vlastně", "vsak", "vy", "vám", "vámi", "vás", "váš", "více", "však", "všechen", "všechno", "všichni", "vůbec", "vždy", "z", "za", "zatímco", "zač", "zda", "zde", "ze", "zpet", "zpravidla", "zpravy", "zprávy", "zpět", "čau", "či", "čí", "článek", "článku", "články", "čtrnáct", "čtyři", "šest", "šestnáct", "že"
]
STOP_WORDS = set(STOP_WORDS_LIST)

# | SETTING WN |

if not any(l.language == 'en' for l in wn.lexicons()):
    try: wn.download('omw-en31:1.4')
    except: pass

CZECH_WN_FILE = 'wn-ntumc-ces 2.xml'
if not any(l.language == 'cs' and l.version == '3.0' for l in wn.lexicons()):
    try: wn.add(CZECH_WN_FILE); print("CS WordNet načten.")
    except Exception as e: print(f"Chyba CS WN: {e}")

# | FUNCTIONS |
def get_english_definition(czech_synset):
    try:
        if czech_synset.ili.id:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                res = wn.synsets(ili=czech_synset.ili.id, lang='en')
            if res: return res[0].definition()
    except: pass
    return "Def not found"

def get_czech_synonyms(synset):
    """Vrátí česká synonyma pro daný synset"""
    lemmas = [w.lemma() for w in synset.words()]
    return ", ".join(lemmas[:5]) # Max 4 synonyms

def get_lemma(word):
    return simplemma.lemmatize(word, lang='cs')

print(" Cell 1: Done. ")

 Cell 1: Done. 


In [ ]:
# | LOADING TEXT FILE |
filename = "benchmark_text.txt"
if os.path.exists(filename):
    with open(filename, "r", encoding="utf-8") as f:
        TEXT = f.read().strip()
    print(f"📄 Text: \"{TEXT[:20]}...\"")
else:
    TEXT = "Na kole se mu udělalo nevolno, asi mu nesedl ten poslední panák... vyměnit duši..."
    print("⚠️ Demo text.")

# | LINE-UP OF MODELS |
MODELS = [
    "mistral-nemo:12b",  
    "qwen2.5:14b",       
    "llama3.1:8b",       
    "gemma2:9b"          
] 
print(f" {len(MODELS)} ready")

📄 Text: "Na kole se mu udělal..."
 4 ready


In [ ]:
results = []
words_raw = TEXT.split()
WINDOW_SIZE = 2

print(f"START ...")

for model_name in MODELS:
    print(f"\n Model: {model_name} running")
    
    for i, word in enumerate(words_raw):
        clean_word = re.sub(r'[^\w]', '', word).lower()
        
        # Skip logic
        if not clean_word or clean_word in STOP_WORDS or clean_word.isdigit(): continue

        # | LOCAL CONTEXT |
        start_idx = max(0, i - WINDOW_SIZE)
        end_idx = min(len(words_raw), i + WINDOW_SIZE + 1)
        local_context = " ".join(words_raw[start_idx:end_idx])
        highlighted_context = local_context.replace(word, f"***{word}***")

        lemma = get_lemma(clean_word)
        candidates = wn.synsets(lemma, lang='cs')
        
        # | A) LOADING POSSIBLE IDs |
        options_text = "| WORDNET SENSES |\n"
        valid_ids = {} 

        if candidates:
            for cand in candidates:
                eng_def = get_english_definition(cand)
                cz_syns = get_czech_synonyms(cand)
                valid_ids[cand.id] = {"def": eng_def, "syn": cz_syns}
                # Formatting model: ID + Synonyms + Definitions (they are from english wordnet, since i don't have these in czech one)
                options_text += f"ID: {cand.id}\n   Synonyms (CZ): {cz_syns}\n   Definition (EN): {eng_def}\n\n"
        else:
            options_text += "(No specific WordNet senses found)\n"
        options_text += "| SPECIAL TAGS (Fallbacks) |\n"
        for tag, desc in SPECIAL_TAGS.items():
            options_text += f"Tag: {tag}  ->  {desc}\n"

        # | PROMPTING THE MODEL |
        prompt = f"""
        You are an expert linguistic system for Czech Word Sense Disambiguation.
        
        TASK DATA:
        - Target Word: "{word}" (Lemma: "{lemma}")
        - Local Context: "... {highlighted_context} ..."
       
        AVAILABLE OPTIONS:
        {options_text}
        
        INSTRUCTIONS:
        1. Analyze the local context and full text to determine the meaning of the target word.
        2. Select the most appropriate option from the list above.
        3. If a WordNet ID fits, output ONLY that ID (e.g., cswn-xxxxxx-x).
        4. If no WordNet ID fits (e.g., proper name, organization, specific location), select the best Special Tag (per, org, loc, dat, num, oth, e).
        5. CRITICAL: Output strictly the ID or Tag string. Do not write explanations. Do not write "Option 1".
        """
        # | TIME COUNTING |
        start_t = time.time()
        try:
            response = ollama.generate(model=model_name, prompt=prompt, options={'temperature': 0}) #Temperature is an important setting for model to not be random and crative.
            output = response['response'].strip()
            elapsed = time.time() - start_t
            
            # | PARSING |
            final_tag = "UNKNOWN"
            final_def = ""
            final_syns = ""
            
            # 1. Finding WordNet ID (cswn-...)
            wn_match = re.search(r'(cswn-[\w-]+)', output)
            
            # 2. Finding Special Tag (per, org...)
            tags_pattern = r'\b(' + '|'.join(SPECIAL_TAGS.keys()) + r')\b'
            tag_match = re.search(tags_pattern, output, re.IGNORECASE)
            if wn_match and wn_match.group(1) in valid_ids:
                final_tag = wn_match.group(1)
                final_def = valid_ids[final_tag]["def"]
                final_syns = valid_ids[final_tag]["syn"]
            elif tag_match:
                final_tag = tag_match.group(1).lower()
                final_def = SPECIAL_TAGS.get(final_tag, "Special Category")
                final_syns = "N/A"
            else:
                # IF regex found nothing, model probably did long reply.
                final_tag = "FORMAT_ERROR"
                final_def = output[:40].replace("\n", " ") # Saves part of response for a debug
                
            # | RESOLUTING (by the gold standart) |
            correct_val = GOLD_STANDARD.get(lemma)
            is_correct = None 
            if isinstance(correct_val, list):
                is_correct = (final_tag in correct_val)
            else:
                is_correct = (final_tag == correct_val)
            results.append({
                "Slovo": word,
                "Lemma": lemma,
                "Model": model_name,
                "Tag": final_tag,
                "Synonyma": final_syns,
                "Definice": final_def,
                "Cas_sec": round(elapsed, 2),
                "Spravne": is_correct,
                "Gold_Tag": correct_val if correct_val else ""
            })
            
        except Exception as e:
            print(f" Chyba {word}: {e}")
print(" DONE.")

START ...

 Model: mistral-nemo:12b running

 Model: qwen2.5:14b running

 Model: llama3.1:8b running

 Model: gemma2:9b running
 DONE.


In [34]:
if results:
    df = pd.DataFrame(results)
    
    # OPRAVA CHYBY (List -> String)
    # Protože Gold_Tag může být seznam ['id1', 'id2'], pandas podle něj neumí seskupovat.
    # Převedeme ho na textový řetězec, např. "id1, id2"
    df['Gold_Tag_Str'] = df['Gold_Tag'].apply(lambda x: ", ".join(x) if isinstance(x, list) else str(x))

    # PIVOT: Detailní pohled (Tag + Synonyma)
    df['View'] = df.apply(lambda r: f"[{r['Tag']}] {r['Synonyma'] if r['Synonyma'] else ''}", axis=1)
    
    # Použijeme 'Gold_Tag_Str' místo 'Gold_Tag' v indexu
    pivot_detail = df.pivot_table(
        index=["Slovo", "Lemma", "Gold_Tag_Str"], 
        columns="Model", 
        values="View", 
        aggfunc="first"
    )
    
    # STATISTIKA PŘESNOSTI (Accuracy)
    df_eval = df[df['Spravne'].notna()]
    if not df_eval.empty:
        accuracy = df_eval.groupby('Model')['Spravne'].mean() * 100
        accuracy = accuracy.sort_values(ascending=False).reset_index()
        accuracy.columns = ['Model', 'Accuracy (%)']
    else:
        accuracy = pd.DataFrame()
    
    # STATISTIKA RYCHLOSTI
    speed = df.groupby('Model')['Cas_sec'].mean().sort_values().reset_index()
    speed.columns = ['Model', 'Avg Time (s/word)']
    
    # SPOJENÍ STATISTIK
    if not accuracy.empty:
        stats = pd.merge(accuracy, speed, on="Model")
    else:
        stats = speed

    print("\n🏆 VÝSLEDKY EXPERIMENTU:")
    display(stats)
    
    print("\n📊 DETAILNÍ POHLED (Slovo -> Model -> Výsledek):")
    display(pivot_detail)

    # Uložení
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    fname = f"wsd_final_{timestamp}.xlsx"
    with pd.ExcelWriter(fname) as writer:
        stats.to_excel(writer, sheet_name='Stats', index=False)
        pivot_detail.to_excel(writer, sheet_name='Detail')
        df.to_excel(writer, sheet_name='Raw_Data', index=False)
        
    print(f"\n💾 Uloženo do: {fname}")
    print("   (Obsahuje listy: Stats, Detail, Raw_Data)")
else:
    print("⚠️ Žádná data.")


🏆 VÝSLEDKY EXPERIMENTU:


,Model,Accuracy (%),Avg Time (s/word)
0,mistral-nemo:12b,91.304348,1.069130
1,gemma2:9b,86.956522,1.005217
2,llama3.1:8b,86.956522,0.914783
3,qwen2.5:14b,86.956522,1.360000



📊 DETAILNÍ POHLED (Slovo -> Model -> Výsledek):


,,Model,gemma2:9b,llama3.1:8b,mistral-nemo:12b,qwen2.5:14b
Slovo,Lemma,Gold_Tag_Str,,,,
Cestou,cesta,"cswn-04096066-n, cswn-04564698-n, cswn-00277962-n","[cswn-04096066-n] cesta, trasa, silnice",[cswn-00312691-n] cesta,"[cswn-04096066-n] cesta, trasa, silnice","[cswn-00306426-n] cesta, putování"
Nakonec,nakonec,cswn-00047641-r,"[cswn-00047641-r] časem, nakonec","[cswn-00047641-r] časem, nakonec","[cswn-00047641-r] časem, nakonec","[cswn-00047641-r] časem, nakonec"
domů,dům,"cswn-03544360-n, cswn-08078020-n, loc","[cswn-03544360-n] obytná budova, obydlí, dům, ...","[cswn-03544360-n] obytná budova, obydlí, dům, ...","[cswn-03544360-n] obytná budova, obydlí, dům, ...","[cswn-03544360-n] obytná budova, obydlí, dům, ..."
"duši,",duše,"cswn-10627082-n, cswn-03573005-n","[cswn-00007846-n] smrtelník, člověk, duše, oso...","[cswn-10627082-n] duch, duše",[cswn-03573005-n] duše,"[cswn-10627082-n] duch, duše"
dělníci.,dělník,cswn-10791221-n,[cswn-10791221-n] dělník,[cswn-10791221-n] dělník,[cswn-10791221-n] dělník,[cswn-10791221-n] dělník
háku,hák,"cswn-03532672-n, cswn-03532342-n","[cswn-03532672-n] háček, skoba, hák",[cswn-03532342-n] hák,"[cswn-03532672-n] háček, skoba, hák","[cswn-03532672-n] háček, skoba, hák"
kole,kolo,"cswn-02834778-n, cswn-02974003-n",[cswn-02974003-n] kolo,"[cswn-02834778-n] kolo, jízdní kolo, bicykl",[cswn-02974003-n] kolo,"[cswn-02834778-n] kolo, jízdní kolo, bicykl"
kouřili,kouřit,cswn-01198101-v,"[cswn-01198101-v] bafat, dýmat, kouřit, čoudit...","[cswn-01198101-v] bafat, dýmat, kouřit, čoudit...","[cswn-01198101-v] bafat, dýmat, kouřit, čoudit...","[cswn-01198101-v] bafat, dýmat, kouřit, čoudit..."
mu,on,"cswn-77000046-n, oth",[cswn-77000046-n] on,[cswn-77000046-n] on,[cswn-77000046-n] on,[cswn-77000046-n] on



💾 Uloženo do: wsd_final_20260130_2211.xlsx
   (Obsahuje listy: Stats, Detail, Raw_Data)
